<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/ufidon/ml/blob/main/mod2/cmte.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
  </td>
  <td>
    <a target="_blank" href="https://kaggle.com/kernels/welcome?src=https://github.com/ufidon/ml/blob/main/mod2/cmte.ipynb"><img src="https://kaggle.com/static/images/open-in-kaggle.svg" /></a>
  </td>
</table>
<br>


__Classification__

_homl3 ch3_

- MNIST - a dataset of handwritten digits
- Building a digit recognizer
- Model evaluation
  - Measuring Accuracy Using Cross-Validation
  - Confusion Matrices
  - Precision and Recall
  - The Precision/Recall Trade-off
  - The ROC Curve
- Multiclass Classification
  - Error Analysis
- Multilabel Classification
- Multioutput Classification

In [ ]:
import numpy as np, pandas as pd, matplotlib.pyplot as plt, matplotlib as mpl
import sklearn as skl, sklearn.datasets as skds

[MNIST - a dataset of handwritten digits](https://en.wikipedia.org/wiki/MNIST_database)
---
- Modified National Institute of Standards and Technology database (MNIST)
- a large database of handwritten digits used by image processing systems
- contains 70,000 black and white images 
  - 60,000 for training and 10,000 for testing
- each image is normalized to fit into a 28x28 pixel bounding box and anti-aliased

In [ ]:
# fetch the dataset from https://www.openml.org/
mnist = skds.fetch_openml('mnist_784', as_frame=False)

# the returned if of type sklearn.utils.Bunch
# this is a dictionary whose keys can also be accessed as attributes
mnist.keys()

In [ ]:
# the description of the dataset
print(mnist.DESCR)

In [ ]:
X,y = mnist.data, mnist.target
X.shape, y.shape

In [ ]:
plt.imshow(X[1000].reshape((28,28))), y[1000]

In [ ]:
fig, axs=plt.subplots(10,10,figsize=(9,9), layout='constrained')
for idx, dimg in enumerate(X_test[:100]):
  axs[idx//10, idx%10].imshow(dimg.reshape((28,28)), cmap='binary')
  axs[idx//10, idx%10].axis("off")


In [ ]:
# The dataset is already shuffled and split into a training set and a test set
X_train, y_train = X[:60_000], y[:60_000]
X_test, y_test = X[60_000:], y[60_000:]
# 👍 Thumb rule for data splitting: 80% for training 20% for testing

Building a digit recognizer
---
- Let's start from recognizing a single digit such as
  - `0` or `non-0`, `8` or `non-8`
  - which is a binary classifier
- can be implemented with many scikit-learn's classifiers, e.g.
  - stochastic gradient descent (SGD, or stochastic GD) classifier
  - implemented in the scikit-learn's SGDClassifier class

In [ ]:
# create and train a binary classifier to recognize 8
from sklearn.linear_model import SGDClassifier
clfSgd = SGDClassifier(random_state=50)
y_train_8 = (y_train == '8')
clfSgd.fit(X_train, y_train_8)

In [ ]:
# recognize 8 from test images using this classifier
res = clfSgd.predict(X_test[:100])
res.reshape((10,10))

In [ ]:
fig1, axs1=plt.subplots(10,10,figsize=(9,9), layout='constrained')
for idx, dimg in enumerate(X_test[:100]):
  axs1[idx//10, idx%10].imshow(dimg.reshape((28,28)), cmap='binary') if res[idx] == False else axs1[idx//10, idx%10].imshow(dimg.reshape((28,28)))
  axs1[idx//10, idx%10].axis("off")

[Model evaluation](https://scikit-learn.org/stable/model_selection.html)
---
- many metrics are available for model evaluation, such as
  - confusion matrix
  - accuracy, precision, recall, f1 score, etc.
- which metrics are preferred depends on the requirements

Measuring Accuracy Using Cross-Validation
---
- k-fold cross-validation
  - split the training set into k folds
  - train the model k times
  - hold out a different fold each time for evaluation
  - implemented with cross_val_score in scikit

accuracy=(# of correct predictions)/(# of all predictions)


In [ ]:
from sklearn.model_selection import cross_val_score
cross_val_score(clfSgd, X_train, y_train_8, cv=5, scoring='accuracy') # cv=5 number of folds, default 5

In [ ]:
# The accuracies are quite good for all folds. 
# However, this is caused by the imbalance of the chosen data.
# by just telling not-8 every time, we get 90% right
1-len(y_train_8[y_train_8 == True])/len(y_train_8)

In [ ]:
# equally randomly guess imbalanced data achieves high accuracy
# so accuracy is NOT useful in situations with highly imbalanced data
from sklearn.dummy import DummyClassifier
clfDummy = DummyClassifier()
clfDummy.fit(X_train, y_train_8)
cross_val_score(clfDummy, X_train, y_train_8, scoring='accuracy')

In [ ]:
# an implementation of cross-validation

from sklearn.model_selection import StratifiedKFold
from sklearn.base import clone

skFolder = StratifiedKFold(n_splits=5, shuffle=True)
for trainIndex, testIndex in skFolder.split(X_train, y_train_8):
  cloneClf = clone(clfSgd)
  X_trainFold = X_train[trainIndex]
  y_trainFold = y_train_8[trainIndex]
  X_testFold = X_train[testIndex]
  y_testFold = y_train_8[testIndex]

  cloneClf.fit(X_trainFold, y_trainFold)
  yPred = cloneClf.predict(X_testFold)
  nCorrect = sum(yPred == y_testFold)
  print(nCorrect/len(yPred), end=" ")

[Confusion Matrices](https://en.wikipedia.org/wiki/Confusion_matrix)
---
- visualize of the performance of algorithms
- show number of misclassifications

| Actual\Prediction | non-`8` | `8` |
|:---:|:---:|:---:|
| non-`8` | True negative (TN) | False positive (FP)<br>or type I error |
| `8` | False negative (FN)<br>or type II error | True positive (TP) |

In [ ]:
# generate the confusion matrix on training data
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix

y_train_pred = cross_val_predict(clfSgd, X_train, y_train_8)
cm = confusion_matrix(y_train_8, y_train_pred)

In [ ]:
cm

Precision, recall and F1 score
---

- the precision of the classifier is the accuracy of the positive predictions


$\displaystyle precision=\frac{TP}{TP+FP}$

- could be misleading in the case like
  - always make negative predictions
  - make only one positive prediction on the instance it is sure about
  - then, precision = 1/1 = 100%
- so, precision is usually used along with *recall*, the *sensitivity*, or the *true positive rate (TPR)*
  - i.e. the ratio of positive instances correctly predicted by the classifier

$\displaystyle recall=\frac{TP}{TP+FN}$

- Now, accuracy can be calculated as

$\displaystyle accuracy = \frac{TP+TN}{P+N}=\frac{TP+TN}{TP+TN+FP+FN}$

In [ ]:
# calculate precision and recall
from sklearn.metrics import precision_score, recall_score, accuracy_score
tn,fp,fn,tp = cm.flatten()
print(f'precision=TP/(TP+FP)={tp}/({tp}+{fp})={tp/(tp+fp)}={precision_score(y_train_8,y_train_pred)}')
print(f'recall=TP/(TP+FN)={tp}/({tp}+{fn})={tp/(tp+fn)}={recall_score(y_train_8,y_train_pred)}')
print(f'accuracy=(TP+TN)/(TP+TN+FP+FN)=({tp}+{tn})/({tp}+{tn}+{fp}+{fn})={cm.trace()/cm.sum()}={accuracy_score(y_train_8,y_train_pred)}')

- the classifier is correct only 60.33% of the time
  - detects 56.79% of the `8`'s
- precision and recall can be combined into a single metric $F_1$ score
  - the *harmonic mean* of precision and recall
  - gives more weight to low values
  - $F_1$ is high when both precision and recall are high

$\displaystyle F_1=\frac{2}{\frac{1}{precision}+\frac{1}{recall}}=\frac{2TP}{2TP+FN+FP}$

In [ ]:
# calculate F1 score
from sklearn.metrics import f1_score
print(f'f1 = 2TP/(2TP+FN+FP)=2*{tp}/(2*{tp}+{fn}+{fp})={2*tp/(2*tp+fn+fp)}={f1_score(y_train_8, y_train_pred)}')

The Precision/Recall Trade-off
---
- the *precision/recall trade-off* is that increasing precision reduces recall, and vice versa
- the SGDClassifier makes its classification decisions in two steps
  - computes a score based on a decision function
  - compares the score with a threshold
    - classifies the instance as positive if score > threshold
    - else negative
  - the default threshold used by the SGDClassifier is 0
  - raising the threshold decreases recall
- the appropriate threshold, i.e. the precision/recall trade-off can be made on
  - the curves of precision vs. threshold and recall vs. threshold

In [ ]:
# calculate all precisions and recalls vs. thresholds
y_scores = cross_val_predict(clfSgd, X_train, y_train_8, method='decision_function')

from sklearn.metrics import precision_recall_curve
precisions, recalls,thresholds = precision_recall_curve(y_train_8, y_scores)

In [ ]:
# draw the curves of precision vs. threshold and recall vs. threshold
threshold = 3000
fig2, ax2 = plt.subplots(figsize=(6,3),layout='constrained')
ax2.plot(thresholds,precisions[:-1], 'b--', label='Precision', linewidth=2)
ax2.plot(thresholds,recalls[:-1],'g-', label='Recall', linewidth=2)
ax2.vlines(threshold,0, 1.0, 'r', "dotted", label='threshold')

idx = (thresholds >= threshold).argmax() # first index ≥ threshold
ax2.plot(thresholds[idx], precisions[idx], 'bo')
ax2.plot(thresholds[idx], recalls[idx], 'go')
ax2.grid('on')
ax2.axis([-50000,25000,-0.01,1.01])
ax2.text(-5000,.05, 'Threshold')
ax2.legend(loc='center right')

- A good precision/recall trade-off can also be chosen on the curve of precision vs. recall

In [ ]:
# select a good threshold on the curve of precision vs. recall
fig3, ax3 = plt.subplots(figsize=(5,4),layout='constrained')
ax3.plot(recalls, precisions, linewidth=2, label='Precision/Recall curve')

ax3.plot([recalls[idx], recalls[idx]], [0, precisions[idx]], 'r:')
ax3.plot([0, recalls[idx]], [precisions[idx], precisions[idx]], 'r:')
ax3.plot([recalls[idx]], [precisions[idx]], 'ro', label=f'Point at threshold {threshold}')
import matplotlib.patches as mpp
ax3.add_patch(mpp.FancyArrowPatch((0.7,0.44), (0.6,0.6), connectionstyle='arc3,rad=.0',
                                  arrowstyle='Simple, tail_width=2, head_width=8, head_length=10',
                                  color='#ff0000'))
ax3.text(.7,.52,'Higher\nthreshold',color='#0000ff')
ax3.axis([0,1,0,1])
ax3.grid()
ax3.set_xlabel('Recalls')
ax3.set_ylabel('Precision')
ax3.legend(loc='lower left')

- a good Precision/Recall (PR) curve bends toward the top-right corner

In [ ]:
# find the threshold giving 90% precision
idx_for_90_precision = (precisions >= 0.90).argmax()
threshold_for_90_precision = thresholds[idx_for_90_precision]
threshold_for_90_precision

In [ ]:
# make predictions and check their precision and recall
y_train_pred_90 = (y_scores >= threshold_for_90_precision)
print(f'precision={precision_score(y_train_8, y_train_pred_90)}\nrecall={recall_score(y_train_8, y_train_pred_90)}')

- With 90% precision, however, the recall is too low to be acceptable

[The ROC Curve](https://en.wikipedia.org/wiki/Receiver_operating_characteristic)
---
- i.e. the receiver operating characteristic (ROC) curve
  - similar to precision vs. recall curve
- plots *true positive rate (TPR)* (another name for recall) vs. *false positive rate (FPR)*
  - at each threshold
- FPR is also called fall-out

$\displaystyle FPR=\frac{FP}{FP+TN}=1-TNR$ 

$\displaystyle TNR=\frac{TN}{FP+TN}$

In [ ]:
# find all fpr and tpr at each threshold
from sklearn.metrics import roc_curve
fpr, tpr, thresholds = roc_curve(y_train_8, y_scores)
idx_for_threshold_at_90 = (thresholds <= threshold_for_90_precision).argmax()
tpr_90, fpr_90 = tpr[idx_for_threshold_at_90], fpr[idx_for_threshold_at_90]

In [ ]:
fig4, ax4 = plt.subplots(figsize=(5,4),layout='constrained')
ax4.plot(fpr, tpr, linewidth=2, label='ROC curve')
ax4.plot([0,1],[0,1], 'c:', label="Random classifier's ROC curve")
ax4.plot([fpr_90], [tpr_90], 'ro', label='Threshold for 90% precision')

ax4.add_patch(mpp.FancyArrowPatch((.25,.75),(.1,.62), connectionstyle='arc3,rad=.2',
                                  arrowstyle='Simple, tail_width=1.5, head_width=8, head_length=10',
                                  color='#ff0000'))
ax4.text(.2,.64,"Higher\nthreshold", color='#0000ff')
ax4.axis([0,1,0,1])
ax4.grid()
ax4.set_xlabel('False Positive Rate (Fall-Out)')
ax4.set_ylabel('True Positive Rate (Recall)')
ax4.legend(loc="lower right")


- the ROC curve of a good classifier bends toward the top-left corner
  - trade-off: the higher TPR, the higher FPR
- the metric for classifier related to ROC is the *area under the curve (AUC)*
  - area of 1 means a perfect classifier
  - area of 0.5 means a purely random classifier

In [ ]:
# calculate the ROC AUC
from sklearn.metrics import roc_auc_score
print(f'roc auc={roc_auc_score(y_train_8, y_scores)}')

Compare SGDClassifier with RandomForestClassifier
---
- classifier can be compared with the evaluation metrics such as
  - Precision/Recall (PR) curve, F1 score, ROC AUC score, etc.
- RandomForestClassifier.predict_proba() calculates class probabilities for each instance
  - the probabilities of the positive class can be used as scores

In [ ]:
# train and evaluate a RandomForestClassifier
from sklearn.ensemble import RandomForestClassifier
forest_clf = RandomForestClassifier(random_state=50)
y_probas_forest = cross_val_predict(forest_clf, X_train, y_train_8, method='predict_proba')

In [ ]:
# show the class probabilities for the first 10 images in the training set
# each row: probability of being negative, probability of being positive
y_probas_forest[:10]

In [ ]:
# ues the estimated probabilities for the positive class as the scores
y_scores_forest = y_probas_forest[:,1]
precisions_forest, recalls_forest, thresholds_forest = precision_recall_curve(y_train_8, y_scores_forest)

In [ ]:
fig5, ax5 = plt.subplots(figsize=(5,4),layout='constrained')
ax5.plot(recalls_forest, precisions_forest, 'b-', linewidth=2, label='Random forest')
ax5.plot(recalls, precisions, '--', linewidth=2, label='SGD')

ax5.set_xlabel('Recall')
ax5.set_ylabel('Precision')
ax5.axis([0,1,0,1])
ax5.grid()
ax5.legend(loc='lower left')

-  RandomForestClassifier's PR curve is much better than SGDClassifier's

In [ ]:
# calculate RandomForestClassifier's f1 score, roc auc score, precision and recall
y_train_pred_forest = y_probas_forest[:, 1] >= 0.5 
print(f'f1 score={f1_score(y_train_8, y_train_pred_forest)}')
print(f'roc auc score={roc_auc_score(y_train_8, y_train_pred_forest)}')
print(f'precision score={precision_score(y_train_8, y_train_pred_forest)}')
print(f'recall score={recall_score(y_train_8, y_train_pred_forest)}')

- they are all better than SGDClassifier's respective scores

Multiclass Classification
---
- binary classifiers distinguish between two classes
  - such as SGDClassifier and SVC
  - multiple binary classifiers can be combined to classify multiple classes
- multiclass classifiers can distinguish between more than two classes
  - also called multinomial classifiers
  - such as RandomForestClassifier, GaussianNB and LogisticRegression


Two Multiclass Classification Strategies used by Binary Classifier (BC)
---
- one-versus-the-rest (OvR)
   - also called one-versus-all (OvA)
   - needs $M$ BCs, one for each class
   - classify an instance with all classifiers and choose the class output from the classifier with the highest score
   - preferred by most BCs
 - one-versus-one (OvO)
   - train a BC for every pair of classes
   - needs $\displaystyle {{M}\choose{2}}=\frac{M(M-1)}{2}$ BCs
   - each classifier only needs to be trained on the part of the training set containing the two classes that it must distinguish
   - mainly used by BCs that scale poorly with the size of the training set
   - rare, such as *support vector machine classifier*
 - scikit runs OvR or OvO automatically for multi-classification with BCs

In [ ]:
# Multi-classify 10 digits with SVC
# scikit used the OvO strategy and trained 45 binary classifiers
from sklearn.svm import SVC

svm_clf = SVC(random_state=50)
svm_clf.fit(X_train[:2000], y_train[:2000]) # choose a small part of the data set

In [ ]:
svm_predicts = svm_clf.predict(X_test[:100])
print(f'{svm_predicts}\n{svm_predicts==np.array(y_test[:100])}')

  - Error Analysis
- Multilabel Classification
- Multioutput Classification

# References
- [Model selection and evaluation in scikit](https://scikit-learn.org/stable/model_selection.html)